# Google Maps API and Python Best Practices

## Import libraries

In [1]:
import os
import math
import folium
import requests
import googlemaps
import pandas as pd
from dotenv import load_dotenv

## Load the API key

> Always use ENVIRONMENT VARIABLES for storing credentials (e.g., API keys, usernames, passwords)

In [2]:
load_dotenv()
api_key = os.getenv('API_KEY')

## Create a Google Maps instance

In [3]:
gmaps = googlemaps.Client(key=api_key)

## Scrape data from DepEd
https://ebeis.deped.gov.ph/beis/reports_info/masterlist

**DepEd Masterlist of Schools Landing Page**

![deped_schools_landing_page](assets/deped_schools_landing_page.png)

**DepEd Masterlist of Schools Inspect Page**

![deped_schools_inspect](assets/deped_schools_inspect.png)

### Convert cURL to a Python request
https://curlconverter.com/

In [4]:
cookies = {
    'sf_beis': 'maqosutmsq5u5h151p0lsljll4',
    'switchmenu-frontend': '',
}

headers = {
    'authority': 'ebeis.deped.gov.ph',
    'accept': 'text/javascript, text/html, application/xml, text/xml, */*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': 'sf_beis=maqosutmsq5u5h151p0lsljll4; switchmenu-frontend=',
    'origin': 'https://ebeis.deped.gov.ph',
    'referer': 'https://ebeis.deped.gov.ph/beis/reports_info/masterlist',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'x-prototype-version': '1.6.0.3',
    'x-requested-with': 'XMLHttpRequest',
}

# This is where you would change the parameters in case you want to get more data
data = {
    'page': '1',
    'school[id]': '',
    'school[school_name]': '',
    'school[co_gen_class]': '',
    'school[general_classification_id]': '',
    'school[curricular_class_id]': '',
    'school[region_id]': '13', # NCR
    'school[division_id]': '1008', # Manila
    'school[user_id]': '',
    'school[school_head]': '',
    'ajax_submit': 'Submit',
    '_': '',
}

response = requests.post(
    'https://ebeis.deped.gov.ph/beis/reports_info/viewMasterList',
    cookies=cookies,
    headers=headers,
    data=data,
)

### Check if the request was successful

200 - success <br/>
403 - error

In [5]:
response.status_code

200

### View the data

In [7]:
schools = pd.read_html(response.text)[0]

In [8]:
row, col = schools.shape
print(f'{row} rows, {col} columns')

30 rows, 5 columns


In [9]:
schools.head()

,School ID,School Name,Head / Position,Address,School Type
0,136418,A. C. Herrera Elementary School,"FABONAN, NICHOL WAYNE FERRIOL - Principal I",S. Teodoro St.,Kinder & Grade 1-6
1,136453,A. Mabini Elementary School,"SAMPANG, RODEL CUNANAN - Principal III","Severino-Soler St.,",Kinder & Grade 1-6
2,136436,A. V. Hernandez Elementary School,"SORIANO, EMMANUEL REYES - Principal IV","Sto. Niño St., Bo. Magsaysay",Kinder & Grade 1-6
3,409082,"A. Veloso Pre-school Foundation, Inc.","AVPSF, AVPSF AVPSF - Administrator",1706 G. Perfecto Street,Kinder
4,401141,ABE International College of Business & Econom...,"ABE COLLEGE-MANILA, ABE COLLEGE-MANILA ABE COL...",2578 Legarda Street,Grade 11-12


### Remove duplicates

In [10]:
# There are some schools with the same school name and address but different school IDs
schools.drop_duplicates(subset=['School Name', 'Address'], ignore_index=True, inplace=True)

In [11]:
schools.head()

,School ID,School Name,Head / Position,Address,School Type
0,136418,A. C. Herrera Elementary School,"FABONAN, NICHOL WAYNE FERRIOL - Principal I",S. Teodoro St.,Kinder & Grade 1-6
1,136453,A. Mabini Elementary School,"SAMPANG, RODEL CUNANAN - Principal III","Severino-Soler St.,",Kinder & Grade 1-6
2,136436,A. V. Hernandez Elementary School,"SORIANO, EMMANUEL REYES - Principal IV","Sto. Niño St., Bo. Magsaysay",Kinder & Grade 1-6
3,409082,"A. Veloso Pre-school Foundation, Inc.","AVPSF, AVPSF AVPSF - Administrator",1706 G. Perfecto Street,Kinder
4,401141,ABE International College of Business & Econom...,"ABE COLLEGE-MANILA, ABE COLLEGE-MANILA ABE COL...",2578 Legarda Street,Grade 11-12


## Geocoding

### Geocode 1 school

**A.C. Herrera Elementary School**

In [12]:
schools[schools['School ID'] == 136418]

,School ID,School Name,Head / Position,Address,School Type
0,136418,A. C. Herrera Elementary School,"FABONAN, NICHOL WAYNE FERRIOL - Principal I",S. Teodoro St.,Kinder & Grade 1-6


In [14]:
ac_herrera = schools.loc[0, 'School Name'] + ' ' + schools.loc[0, 'Address']
ac_herrera

'A. C. Herrera Elementary School S. Teodoro St.'

In [15]:
ac_herrera_response = gmaps.geocode(ac_herrera)
ac_herrera_response

[{'address_components': [{'long_name': 'JXMJ+M4W',
    'short_name': 'JXMJ+M4W',
    'types': ['plus_code']},
   {'long_name': 'Tondo',
    'short_name': 'Tondo',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Manila',
    'short_name': 'Manila',
    'types': ['locality', 'political']},
   {'long_name': 'Metro Manila',
    'short_name': 'NCR',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Philippines',
    'short_name': 'PH',
    'types': ['country', 'political']}],
  'formatted_address': 'JXMJ+M4W, Tondo, Manila, Metro Manila, Philippines',
  'geometry': {'location': {'lat': 14.634225, 'lng': 120.9803436},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 14.6355700802915,
     'lng': 120.9816684802915},
    'southwest': {'lat': 14.6328721197085, 'lng': 120.9789705197085}}},
  'partial_match': True,
  'place_id': 'ChIJHxQw9yy1lzMR47tmIabjf7Q',
  'types': ['establishment', 'point_of_interes

In [16]:
ac_herrera_location = ac_herrera_response[0]['geometry']['location']
ac_herrera_location

{'lat': 14.634225, 'lng': 120.9803436}

**A. Mabini Elementary School**

In [17]:
schools[schools['School ID'] == 136453]

,School ID,School Name,Head / Position,Address,School Type
1,136453,A. Mabini Elementary School,"SAMPANG, RODEL CUNANAN - Principal III","Severino-Soler St.,",Kinder & Grade 1-6


In [18]:
a_mabini = schools.loc[1, 'School Name'] + ' ' + schools.loc[1, 'Address']
a_mabini

'A. Mabini Elementary School Severino-Soler St.,'

In [19]:
a_mabini_response = gmaps.geocode(a_mabini)
a_mabini_response

[{'address_components': [{'long_name': '1001',
    'short_name': '1001',
    'types': ['street_number']},
   {'long_name': 'Severino Street',
    'short_name': 'Severino St',
    'types': ['route']},
   {'long_name': 'Quiapo',
    'short_name': 'Quiapo',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Manila',
    'short_name': 'Manila',
    'types': ['locality', 'political']},
   {'long_name': 'Metro Manila',
    'short_name': 'NCR',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Philippines',
    'short_name': 'PH',
    'types': ['country', 'political']},
   {'long_name': '1001', 'short_name': '1001', 'types': ['postal_code']}],
  'formatted_address': '1001 Severino St, Quiapo, Manila, 1001 Metro Manila, Philippines',
  'geometry': {'location': {'lat': 14.6014479, 'lng': 120.9859112},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 14.6028625802915,
     'lng': 120.9871028802915},
    'southwest':

> Notice that there are more address components

In [20]:
a_mabini_location = a_mabini_response[0]['geometry']['location']
a_mabini_location

{'lat': 14.6014479, 'lng': 120.9859112}

### Create a helper function for identifying address components

In [21]:
a_mabini_address = pd.json_normalize(a_mabini_response[0]['address_components'])
a_mabini_address

,long_name,short_name,types
0,1001,1001,[street_number]
1,Severino Street,Severino St,[route]
2,Quiapo,Quiapo,"[political, sublocality, sublocality_level_1]"
3,Manila,Manila,"[locality, political]"
4,Metro Manila,NCR,"[administrative_area_level_1, political]"
5,Philippines,PH,"[country, political]"
6,1001,1001,[postal_code]


In [22]:
a_mabini_address[a_mabini_address['types'].apply(lambda x: 'sublocality' in x)]['long_name'].values[0]

'Quiapo'

In [23]:
def get_address_components(df: pd.DataFrame):
    try:
        district = df[df['types'].apply(lambda x: 'sublocality' in x)]['long_name'].values[0]
    except:
        district = None
    try:
        city = df[df['types'].apply(lambda x: 'locality' in x)]['long_name'].values[0]
    except:
        city = None
    try:
        region = df[df['types'].apply(lambda x: 'administrative_area_level_1' in x)]['long_name'].values[0]
    except:
        region = None
    try:
        country = df[df['types'].apply(lambda x: 'country' in x)]['long_name'].values[0]
    except:
        country = None
    return (district, city, region, country)

In [24]:
get_address_components(a_mabini_address)

('Quiapo', 'Manila', 'Metro Manila', 'Philippines')

### Geocode all schools

In [25]:
for index in range(len(schools)):
    
    # Concatenate School Name and Address to get the most specific address possible
    complete_address = schools.loc[index, 'School Name'] + ' ' + schools.loc[index, 'Address']
    
    # Geocode the complete address
    gmaps_response = gmaps.geocode(complete_address)
    
    # Ensure that there is a response
    if gmaps_response != []:
        location = gmaps_response[0]['geometry']['location']
        #if location != []:
        # Get the required values
        lat = location['lat']
        lng = location['lng']

        # Place the required values into the dataframe
        schools.loc[index, 'Latitude'] = lat
        schools.loc[index, 'Longitude'] = lng
                   
        address = pd.json_normalize(gmaps_response[0]['address_components'])
        #if address != []:
        # Get the required values
        district, city, region, country = get_address_components(address)        

        # Place the required values into the dataframe
        schools.loc[index, 'District'] = district
        schools.loc[index, 'City'] = city
        schools.loc[index, 'Region'] = region
        schools.loc[index, 'Country'] = country

In [26]:
schools

,School ID,School Name,Head / Position,Address,School Type,Latitude,Longitude,District,City,Region,Country
0,136418,A. C. Herrera Elementary School,"FABONAN, NICHOL WAYNE FERRIOL - Principal I",S. Teodoro St.,Kinder & Grade 1-6,14.634225,120.980344,Tondo,Manila,Metro Manila,Philippines
1,136453,A. Mabini Elementary School,"SAMPANG, RODEL CUNANAN - Principal III","Severino-Soler St.,",Kinder & Grade 1-6,14.601448,120.985911,Quiapo,Manila,Metro Manila,Philippines
2,136436,A. V. Hernandez Elementary School,"SORIANO, EMMANUEL REYES - Principal IV","Sto. Niño St., Bo. Magsaysay",Kinder & Grade 1-6,14.619134,120.960874,"107, Tondo",Manila,Metro Manila,Philippines
3,409082,"A. Veloso Pre-school Foundation, Inc.","AVPSF, AVPSF AVPSF - Administrator",1706 G. Perfecto Street,Kinder,14.615034,120.972026,Tondo,Manila,Metro Manila,Philippines
4,401141,ABE International College of Business & Econom...,"ABE COLLEGE-MANILA, ABE COLLEGE-MANILA ABE COL...",2578 Legarda Street,Grade 11-12,14.600425,120.996037,Sampaloc,Manila,Metro Manila,Philippines
5,401135,ABE International College of Business and Acco...,"ABE COLLEGE-TAFT, ABE COLLEGE-TAFT ABE COLLEGE...",West East Center Bldg. 1336 Taft Ave. cor. Pad...,Grade 11-12,14.579664,120.986584,Ermita,Manila,Metro Manila,Philippines
6,405674,Access Computer and Technical College-Manila C...,"ROS, ROSEMARIE M - Administrator","2nd Floor, Access Bldg., C. M. Recto cor. Quez...",Grade 11-12,14.604614,120.977963,Binondo,Manila,Metro Manila,Philippines
7,405679,"ACLC College of Manila, Inc.","AMA CLC-MORAYTA, AMA CLC-MORAYTA AMA CLC-MORAY...",2355 Legarda corner Manrique Streets,Grade 11-12,14.600770,120.995174,None,Manila,Metro Manila,Philippines
8,406359,Adamson University,ELMAR MONTARIL ABAYA - Principal,"900 San Marcelino St., Ermita","Kinder, Grade 1-6, Grade 7-10 & Grade 11-12 at...",14.587478,120.985616,Ermita,Manila,Metro Manila,Philippines
9,408536,Adonai Conservative Baptist Academy. Inc.,ANTONIO IBARRA SAMONTE JR - School Administrator,"2361-A Laura St., Pandacan, Manila",Kinder & Grade 1-6,14.589693,121.006223,860 Pandacan manila,Manila,Metro Manila,Philippines


### Plot the coordinates

In [27]:
# Create a base map
mapa = folium.Map()

In [28]:
mapa

### Plot the schools

In [29]:
for index, elements in schools.iterrows():
    if math.isnan(elements['Latitude']) or math.isnan(elements['Longitude']):
        continue
    else:
        folium.Marker(
            location=(elements['Latitude'], elements['Longitude']),
            popup=elements['School Name'],
            tooltip='click'
        ).add_to(mapa)

In [30]:
mapa

## Reverse geocoding

In [31]:
ac_herrera_location

{'lat': 14.634225, 'lng': 120.9803436}

In [35]:
gmaps.reverse_geocode(ac_herrera_location)

[{'address_components': [{'long_name': 'JXMJ+M4W',
    'short_name': 'JXMJ+M4W',
    'types': ['plus_code']},
   {'long_name': 'Tondo',
    'short_name': 'Tondo',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Manila',
    'short_name': 'Manila',
    'types': ['locality', 'political']},
   {'long_name': 'Metro Manila',
    'short_name': 'NCR',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Philippines',
    'short_name': 'PH',
    'types': ['country', 'political']}],
  'formatted_address': 'JXMJ+M4W, Tondo, Manila, Metro Manila, Philippines',
  'geometry': {'location': {'lat': 14.634225, 'lng': 120.9803436},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 14.6355739802915,
     'lng': 120.9816925802915},
    'southwest': {'lat': 14.6328760197085, 'lng': 120.9789946197085}}},
  'place_id': 'ChIJHxQw9yy1lzMR47tmIabjf7Q',
  'types': ['establishment', 'point_of_interest', 'school']},
 {'addres